In [1]:
import numpy as np
np.random.seed(2671)  # for reproducibility

import copy
import json
import cv2

from keras.models import load_model
from keras.utils import np_utils
from keras import backend as K
import sys
import operator

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5103)
/Library/Python/2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
catmapfilename='breed_map.list'
catmapfile = open(catmapfilename,'r')
categoryMap = dict()
for line in catmapfile.readlines():
    ls = line.split()
    category = ls[0]
    breed = ls[1]
    categoryMap[category] = breed

categoryIndex = dict(zip(categoryMap.keys(),range(len(categoryMap))))
nb_classes = len(categoryMap)

In [3]:
input_photos = []
breedlist = []
posfilename = "positives_shuf.txt"
inputdirname = "CroppedDogs/"
posfile = open(posfilename,'r')
for line in posfile.readlines():
    infile = line.strip()
    ls0 = infile.split("/")
    ls1 = ls0[1].split("_")
    category = ls1[0]
    img = cv2.resize(cv2.imread(infile), (299, 299), cv2.INTER_LINEAR)
    input_photos.append(img)
    breedlist.append(categoryIndex[category])

In [4]:
print len(input_photos),len(breedlist)
batch_size = 10
nb_epoch = 5

train = np.array(input_photos)
labels = np.array(breedlist)

del input_photos[:]

print train.shape

nToUse = len(train)
testFraction = 1.0/6.0
nTest = int(testFraction*nToUse)

X_test = train[:nTest]
y_test = labels[:nTest]

data_augmentation = False

# convert class vectors to binary class matrices
Y_test = np_utils.to_categorical(y_test, nb_classes)

# input image dimensions
img_rows, img_cols = X_test.shape[1], X_test.shape[2]

if K.image_dim_ordering() == 'th':
    X_test = np.swapaxes(X_test,1,3)
    X_test = np.swapaxes(X_test,2,3)
    input_shape = (X_test.shape[1], img_rows, img_cols)
else:
	input_shape = (img_rows, img_cols, X_test.shape[3])

print input_shape

X_test = X_test.astype('float32')
X_test /= 255

22126 22126
(22126, 299, 299, 3)
(3, 299, 299)


In [5]:
model = load_model("dog_cnn_inception.1.10.keras")

In [6]:
#score = model.evaluate(X_test, Y_test, verbose=0)
Y_pred = model.predict(X_test)

In [9]:
N = 1
total = 0
correct = 0
print Y_pred.shape
testbreedlist = y_test.tolist()
for i,breed in enumerate(testbreedlist):
    result = Y_pred[i,:].tolist()
    sorted_x = sorted(zip(range(len(result)),result), key=operator.itemgetter(1),reverse=True)
    sorted_xN = sorted_x[:N]
    total += 1
    if breed in dict(sorted_xN).keys(): correct += 1

(3687, 120)


In [8]:
print N,total,correct,float(correct)/float(total)

5 3687 3330 0.903173311635


In [10]:
print N,total,correct,float(correct)/float(total)

1 3687 2705 0.733658801193
